In [20]:
import pandas as pd 
import duckdb 

In [21]:
conn = duckdb.connect()

In [ ]:
prefix = "../data/input/dw_legacy_medicare_00_16/adm"
years = range(2000, 2017) 
results = []

for year in years:
    conn.execute(f"""
    CREATE OR REPLACE TABLE adm_data AS
    SELECT * FROM read_parquet('{prefix}_{year}.parquet');
    """)

    # Query to find duplicates
    query = f"""
    SELECT COUNT(*) AS duplicate_count
    FROM (
        SELECT adm_id, bene_id, admission_date, diagnoses, discharge_date, year, admission_source,
               COUNT(*) AS cnt
        FROM adm_data
        GROUP BY ALL
        HAVING COUNT(*) > 1
    ) AS duplicates
    """

    duplicate_count = conn.execute(query).fetchone()[0]
    results.append((year, duplicate_count))

    conn.execute("DROP TABLE adm_data;")

df_results = pd.DataFrame(results, columns=["Year", "Duplicate_Count"])

# results
print("Year-wise Duplicate Counts:")
print(df_results)

conn.close()